## This script uses Mani's TIMPS data (https://docs.google.com/document/d/1aAjcnRubP0HV8hbdqYfNSSnzMediNNgUlP27cOhq2d8/edit) to calculate and plot storm-relative motion (SRM) vectors for each dropsonde from each CPEX-CV convective case that is a TIMPS-tracked MCS.

In [1]:
import os
import sys
import math
import h5py
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm  #to get python's normal library of colormaps
import matplotlib.colors as mplc

import cartopy.crs as ccrs
import cartopy.feature as cfeature
#from cartopy.util import add_cyclic_point
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from datetime import datetime
from datetime import timedelta

import metpy.calc as mpcalc
from metpy.units import units

from PIL import Image
import icartt            #needed to read .ict files

import time

tstart = time.time()


In [2]:
timps_test_path = os.path.join(os.getcwd(), 'TIMPS_data', 'TIMPS_0230041_202209060430_16_-24.nc')

testds = xr.open_dataset(timps_test_path)
testds


<xarray.Dataset>
Dimensions:         (time: 30)
Coordinates:
  * time            (time) datetime64[ns] 2022-09-06T04:30:00 ... 2022-09-06T...
Data variables: (12/34)
    datetime        (time) int64 ...
    centrallat      (time) float64 ...
    centrallon      (time) float64 ...
    centlatwgt      (time) float64 ...
    centlonwgt      (time) float64 ...
    LST             (time) int64 ...
    ...              ...
    propspd         (time) float64 ...
    propdir         (time) float64 ...
    propspdw        (time) float64 ...
    propdirw        (time) float64 ...
    cent_over_land  (time) uint64 ...
    gmd             (time) int64 ...
Attributes:
    long_name:    Precipitation feature information
    description:  Information for a precipitation feature tracked by FiT algo...
    history:      Created Fri Feb 24 01:07:26 2023
    SystemID:     230041
    within_TC:    False

In [4]:
#To calculate total mean propagation speed and direction for a given TIMPS-tracked MCS:
timps_test_path2 = os.path.join(os.getcwd(), 'TIMPS_data', 'propvecw_mag_Sept2022.nc')  #download from https://www.inscc.utah.edu/~mani/cpex_cv/lifetime_prop/2022/09/
timps_test_path3 = os.path.join(os.getcwd(), 'TIMPS_data', 'propvecw_dir_Sept2022.nc')  #download from https://www.inscc.utah.edu/~mani/cpex_cv/lifetime_prop/2022/09/

#dimensions n_mcs = # of MCSs in the given month selected
    #the MCSs are indexed in chronological order, with the associated MCS IDs 
    #being found (in chronological order) at https://www.inscc.utah.edu/~mani/cpex_cv/TIMPS/2022/09/
testds2 = xr.open_dataset(timps_test_path2)
testds3 = xr.open_dataset(timps_test_path3)

print ('MCS Propagation Speed (m/s):', testds2.propvecw_mag[1071].item())            #MCS ID 230041 (matches the MCS in testds above)
print ('MCS Propagation Direction (deg, from):', testds3.propvecw_dir[1071].item())  #MCS ID 230041 (matches the MCS in testds above)
    #How do we know that index 1071 matches MCS ID 230041? The data in 'propvecw_mag_Sept2022.nc' and 'propvecw_dir_Sept2022.nc' is not
        #labeled by the MCS ID (sigh), but is given in the order that the MCS IDs show up on https://www.inscc.utah.edu/~mani/cpex_cv/TIMPS/2022/09/.
        #So, copy/paste all the text from https://www.inscc.utah.edu/~mani/cpex_cv/TIMPS/2022/09/ into an Excel spreadsheet, delete the
        #header rows so that just the MCS ID rows remain, then search for the desired MCS ID and note its Excel row number.
        #The index for the desired MCS ID in 'propvecw_mag_Sept2022.nc' and 'propvecw_dir_Sept2022.nc' will be that row number MINUS 1 (because Python is a 0-based indexing system)

        #There is probably an easier way to find the index from https://www.inscc.utah.edu/~mani/cpex_cv/TIMPS/2022/09/ using
            #Python web scraping, but that requires more in-depth coding that I felt would take more time than the manual way described above

testds2


5.710055276472772


<xarray.Dataset>
Dimensions:       (n_mcs: 6277)
Dimensions without coordinates: n_mcs
Data variables:
    propvecw_mag  (n_mcs) float64 ...
Attributes:
    var_descr:  Propagation vector connecting wgt centroid of start and end p...
    units:      m/s
    region:     cpex_cv
    year:       2022
    month:      09

In [ ]:
#To calculate total mean propagation speed and direction for a given MCS that is NOT tracked by TIMPS:
    #1. Calculate the vector (components) connecting the weighted centroids of the MCS start and end positions (i.e., delta lon/x and delta lat/y and convert to meters) 
    #2. Calculate propagation magnitude/speed using distance / time = sqrt(deltaX^2 + deltaY^2) / MCS lifetime
    #3. Calculate propagation direction (coming FROM) using np.degrees(np.arctan2(-deltax, -deltay))

#example calculating the storm motion vector (propagation speed and direction) for one of the
    #CPEX-CV isolated and scattered cases (no TIMPS IDs available for these cases)
start_lat = 9.67
end_lat = 10.11
start_lon = -18.18
end_lon = -20.23

#get distance (in km) and direction storm is coming from (see "Course 2-1" in http://edwilliams.org/gccalc.htm) from http://edwilliams.org/gccalc.htm
    #can calculate propagation speed using (distance * 1000) / (3600 * TIMPS MCS lifespan (in hours))

#OR, use this (more work though)

#get deltax (deltay) below from http://edwilliams.org/gccalc.htm by holding latitude (longitude) constant while calculating deltax (deltay)
    #when calculating deltax using http://edwilliams.org/gccalc.htm, input the same latitude for lat1 and lat2 that is the mean latitude of the TIMPS MCS
        #(i.e., (start_lat + end_lat) / 2)
deltax = -224.836e3    #should be positive (negative) if the TIMPS MCS moves east (west) with time (meters)
deltay = 48.667e3      #should be positive (negative) if the TIMPS MCS moves north (south) with time (meters)
lifespan = 3600 * 8    #lifespan of the total life of the sampled TIMPS convective system (seconds)
shear_wdir = np.degrees(np.arctan2(-deltax, -deltay))  #(-x, -y)
if shear_wdir < 0:
    shear_wdir += 360
shear_mag = np.sqrt(deltax**2 + deltay**2)   #should be ~5.8 m/s for the given deltay and deltax variables above

print ('MCS Distance Traveled (km):', shear_mag / 1000)
print ('MCS Propagation Speed (m/s):', np.round((shear_mag / lifespan), 1))   #propagation speed (m/s)
print ('MCS Propagation Direction (deg, from):', np.round(shear_wdir, 1))     #propagation direction (deg, from)


In [5]:
###the only variables you (Ben) need to change for the rest of this script are the 2 in this cell.
    ###For Giselle and others, you will need to change much more, including filepaths and downloading IMERG/ERA5 data to make the plots

pressures_to_plot_stream = [950, 800]   #desired pressure levels to be plotted for streamlines
#pressures_to_plot_conv = [950, 800]    #desired pressure levels to be plotted for convergence
pressures_to_plot_RH = [950, 800]       #desired pressure levels to be plotted for RH

#dict of desired flight dates (key) to plot ERA5 streamlines/convergence for and
#their associated list of desired UTC hours to be plotted (e.g., convective case hours/time range)

#CPEX-CV convective cases (average time for dropsondes for a given convective case, per CPEX-CV Well Documented Convection.docx)
case_dict_stream = {'20220906': [12, 15, 17],    #Case 8, 9, and 10, respectively
                   '20220907': [14, 16],         #Case 11, and 12/13, respectively
                   '20220909': [17, 19],         #Case 1 and 2, respectively (no TIMPS IDs for these cases)
                   '20220910': [16, 19, 21],     #Case 23, 3, and 4, respectively (no TIMPS IDs for these cases)
                   '20220914': [12, 14],         #Case 14 and 15, respectively
                   '20220916': [15, 16, 18],     #Case 16, 5, and 17, respectively
                   '20220920': [7, 9],           #Case 6 and 7, respectively
                   '20220922': [7],              #Case 18
                   '20220923': [12],             #Case 19
                   '20220926': [9],              #Case 20
                   '20220929': [12],             #Case 21
                   '20220930': [11, 14]}         #Case 24 and 22, respectively

#case_dict_conv = case_dict_stream
#case_dict_stream = {'20220926': [9]}

# #CPEX and CPEX-AW convective cases
# case_dict_stream = {'20170601': [18, 19, 20, 21, 22],      
#              '20170602': [18, 19, 20, 21, 22],
#              '20170606': [19, 20, 21, 22],
#              '20170610': [20, 21, 22, 23],
#              '20170611': [17, 18, 19, 20, 21, 22],
#              '20170615': [19, 20, 21],
#              '20170616': [19, 20, 21, 22],
#              '20170617': [19, 20, 21, 22],
#              '20170619': [17, 18, 19, 20, 21, 22, 23],
#              '20170620': [17, 18, 19, 20, 21, 22],
#              '20170624': [18, 19, 20, 21, 22],
#              '20210821': [22, 23],
#              '20210822': [0, 1, 2],
#              '20210824': [18, 19, 20, 21]}

# #CPEX(-AW) cases 13 and 16
# case_dict_stream = {'20170611': [14, 15, 16, 17, 18, 19, 20, 21, 22],
#                   '20210824': [15, 16, 17, 18, 19, 20, 21]}
# case_dict_conv = {'20170611': [14, 15, 16, 17, 18, 19, 20, 21, 22],
#                   '20210824': [15, 16, 17, 18, 19, 20, 21]}

# #for testing
#case_dict_stream = {'20220906': [12]}   #normal TIMPS and dropsonde data (as opposed to the 2 cases below)
#case_dict_stream = {'20220920': [9]}   #for testing (has N/A TIMPS IDs)
#case_dict_stream = {'20220916': [18]}  #for testing (has N/A TIMPS IDs and has dropsondes with no GPS/winds)


In [8]:
#set some baseline plot displays

#matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
matplotlib.rcParams['axes.labelsize'] = 18
matplotlib.rcParams['axes.titlesize'] = 18
matplotlib.rcParams['axes.labelweight'] = 'bold'
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['xtick.labelsize'] = 16
matplotlib.rcParams['ytick.labelsize'] = 16
matplotlib.rcParams['legend.fontsize'] = 16
#matplotlib.rcParams['legend.facecolor'] = 'w'
#matplotlib.rcParams['axes.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'
matplotlib.rcParams['hatch.linewidth'] = 0.3

#file_date is the date on which the desired flight took place
#utc_hours_to_plot is the desired UTC hours to be plotted for the given flight (e.g., convective case hours/time range)
for file_date, utc_hours_to_plot in case_dict_stream.items():

    print (file_date + ' streamline plots in progress...')
    
    ###get locations of the dropsonde/Navigation/ERA5 folder and read the appropriate files in
    day_folder = os.path.join(os.getcwd(), file_date)

    #dropsonde data
    drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
    drop_csv = pd.read_csv(drop_csv_path)

    if file_date[:4] == '2017':
        campaign = 'CPEX'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
    elif file_date[:4] == '2021':
        campaign = 'CPEXAW'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
    elif file_date[:4] == '2022':
        campaign = 'CPEXCV'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
    else:
        pass

    #ERA5 data
    era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
    era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure.nc')
    ds = xr.open_dataset(era5_path)

    #Navigation data
    nav_folder = os.path.join(day_folder, 'Nav_files')

    for x in os.listdir(nav_folder):
        if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
            os.remove(os.path.join(nav_folder, x))

    nav_ict_path = os.path.join(nav_folder, os.listdir(nav_folder)[0])  #only one nav file per flight

    if campaign != 'CPEX':  #campaign either CPEXAW or CPEXCV

        nav_ict = icartt.Dataset(nav_ict_path)    #open the ict file with icartt
        flight_lat = nav_ict.data["Latitude"]     #nav latitude, just a normal 1-D array
        flight_lon = nav_ict.data["Longitude"]    #nav longitude, just a normal 1-D array

    else:  #for CPEX navigation files, open the CSV file with pandas
           #(Navigation files for CPEX (2017) are originally .kmz not .ict,
           #so I converted them to CSV for free using https://www.gpsvisualizer.com/convert_input

        nav_ict = pd.read_csv(nav_ict_path)       #open the ict file with pandas instead
        flight_lat = nav_ict["latitude"].values   #nav latitude, just a normal 1-D array
        flight_lon = nav_ict["longitude"].values  #nav longitude, just a normal 1-D array    

    #flight track lat/lon extent [West,East,South,North] for plotting, giving an XX degree buffer around the flight track   
    campaign_extent = [flight_lon.min() - 2.5, flight_lon.max() + 2.5, flight_lat.min() - 2.5, flight_lat.max() + 2.5]


    ###calculate each near-storm dropsonde's mean lat/lon, mean low-level (975 hPa – 925 hPa), 
    ###mean mid-level (900 hPa – 700 hPa), and mean low-up-to-mid-level (975 hPa – 700 hPa) wind vectors,
    ###along with storm-relative motion vector (using TIMPS mean propagation vector)
    ###and add them all to a list and array to be plotted
    
    df_drop = pd.read_csv(drop_metric_filepath)
    df_drop_use = df_drop[df_drop['Date'] == int(file_date)].copy()
    
    drop_data_to_plot = np.empty((len(df_drop_use), 7), dtype = object)

    for x in range(len(df_drop_use)):
        
        date = str(df_drop_use['Date'].iloc[x])
        time0 = str(df_drop_use['Time'].iloc[x]).zfill(6)
        sonde_datetime = date[:4] + '-' + date[4:6] + '-' + date[6:] + ' ' + time0[:2] + ':' + time0[2:4] + ':' + time0[4:]
        
#         #skip sondes without a TIMPS ID (i.e., dont add these sondes to drop_data_to_plot)
#         if pd.isna(df_drop_use['TIMPS ID'].iloc[x]):
#             print (f'Skipping {sonde_datetime} sonde due to TIMPS ID = N/A')
#             continue
        
        if df_drop_use['Environment Falling In'].iloc[x] == 'In Precip':
            print (f'Skipping {sonde_datetime} sonde due to it being "In Precip" (In Precip sondes cannot be inflow sondes)')
            continue
        
        #convert TIMPS mean propagation speed and direction to vector components
        TIMPS_spd = df_drop_use['TIMPS ID Propagation Speed [m/s]'].iloc[x] * units('m/s')
        TIMPS_dir = df_drop_use['TIMPS ID Propagation Direction (from) [deg]'].iloc[x] * units.deg
        TIMPS_u, TIMPS_v = mpcalc.wind_components(TIMPS_spd, TIMPS_dir)  #returns u,v values in whatever unit TIMPS_spd is in
        TIMPS_u = TIMPS_u.m  #get rid of the Pint units
        TIMPS_v = TIMPS_v.m  #get rid of the Pint units

        #calculate the dropsonde's mean lat/lon (no lats/lons if the dropsonde doesn't have any wind data)
        drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
        drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
        drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()
        
        if pd.isna(drop_mean_lon) or pd.isna(drop_mean_lat):
            print (f'Skipping {sonde_datetime} sonde due to no GPS data (and thus no wind data)')
            continue
        
        #calculate the dropsonde's mean-layer low-level wind components, along with mean-layer low-level SRMs
        drop_csv_low = drop_csv_use[(drop_csv_use['Pressure [mb]'] <= 975) & (drop_csv_use['Pressure [mb]'] >= 925)].copy()
        drop_mean_u_low = drop_csv_low['U Comp of Wind [m/s]'].mean()
        drop_mean_v_low = drop_csv_low['V Comp of Wind [m/s]'].mean()
        
        if pd.isna(drop_mean_u_low) or pd.isna(drop_mean_v_low):
            SRM_low_u = None
            SRM_low_v = None
        else:   #SRM = dropsonde mean wind vector minus CO motion vector
            SRM_low_u = drop_mean_u_low - TIMPS_u
            SRM_low_v = drop_mean_v_low - TIMPS_v
        
        #calculate the dropsonde's mean-layer mid-level wind components, along with mean-layer mid-level SRMs
        drop_csv_mid = drop_csv_use[(drop_csv_use['Pressure [mb]'] <= 900) & (drop_csv_use['Pressure [mb]'] >= 700)].copy()
        drop_mean_u_mid = drop_csv_mid['U Comp of Wind [m/s]'].mean()
        drop_mean_v_mid = drop_csv_mid['V Comp of Wind [m/s]'].mean()

        if pd.isna(drop_mean_u_mid) or pd.isna(drop_mean_v_mid):
            SRM_mid_u = None
            SRM_mid_v = None
        else:   #SRM = dropsonde mean wind vector minus CO motion vector
            SRM_mid_u = drop_mean_u_mid - TIMPS_u
            SRM_mid_v = drop_mean_v_mid - TIMPS_v
        
        sonde_info = [drop_mean_lon, drop_mean_lat, time0[:4], SRM_low_u, SRM_low_v, 
                      SRM_mid_u, SRM_mid_v]
        #drop_data_to_plot.append(sonde_info)
        drop_data_to_plot[x, :] = sonde_info
        
    #mask dropsonde rows (i.e., all values of None) that didn't have an associated TIMPS ID (may not be necessary with quiver plotting)
    #drop_data_to_plot = np.ma.masked_where(drop_data_to_plot == None, drop_data_to_plot)
    
        
    ###create an XX-panel plot of streamlines at XX-different pressure levels 
        ###for each desired hour of the given day

    for hr in utc_hours_to_plot:
        
        hr2 = str(hr).zfill(2)
        hour_prior = str(hr - 1).zfill(2)

        #MIMIC TPW data
           ##https://bin.ssec.wisc.edu/pub/mtpw2/data/
        tpw_folder = os.path.join(day_folder, 'MIMIC_TPW_files')
        tpw_path = os.path.join(tpw_folder, 'comp' + file_date + '.' + hr2 + '0000.nc')
        ds_tpw = xr.open_dataset(tpw_path)

        #GPM IMERG data (see IMERG.ipynb for more how to more generally download and plot IMERG data)
           ##https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python
           ##https://disc.gsfc.nasa.gov/datasets?keywords=imerg&page=1
           #0.1 x 0.1 gridded data, half-hourly means, using the half hour BEFORE the desired hour
        imerg_folder = os.path.join(day_folder, 'IMERG_files')
        
        for x in os.listdir(imerg_folder):
            if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
                os.remove(os.path.join(imerg_folder, x))
                
            #minutes and seconds automatically revert to zero (hour = 0, seconds = 0) for '%Y%m%d%H'
            elif (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(seconds = 1), '%H%M%S') in x) and (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(minutes = 30), '%H%M%S') in x):
                imerg_file = x
                break
            else:
                imerg_file = 'Could not find the desired IMERG file'
        
        #confirm that the IMERG file is from the correct day (if hr2 == '00', then this will be the previous day)
        assert (file_date in imerg_file) or (hr2 == '00'), 'IMERG file not from the correct day'
        
        imerg_path = os.path.join(imerg_folder, imerg_file)
        ds_imerg = h5py.File(imerg_path, 'r')
        
        imerg_lons = ds_imerg['Grid/lon'][:]   #Longitude Shape: (3600,)
        imerg_lats = ds_imerg['Grid/lat'][:]   #Latitude Shape: (1800,)
        imerg_lons, imerg_lats = np.meshgrid(imerg_lons, imerg_lats)  #Long and lat grid shape: (1800, 3600) 
        
        imerg_precip = ds_imerg['Grid/precipitation'][0][:][:]  #Original Precip Shape: (1, 3600, 1800) = (time, lon, lat)
        imerg_precip = np.transpose(imerg_precip)               #New Precip Shape after transpose: (1800, 3600)
        
        #mask blank data
        imerg_precip_masked = np.ma.masked_where(imerg_precip < 0, imerg_precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
        imerg_precip_masked = np.ma.masked_where(np.isnan(imerg_precip_masked), imerg_precip_masked)  #masks NaN values (not masked in previous line)        
        
        data_proj = ccrs.PlateCarree()

        group_fig = plt.figure(figsize = (24, 12))   #initialize the streamline figure for the given hour

        for ii, pres_lev in enumerate(pressures_to_plot_stream):
            ax = group_fig.add_subplot(1, 2, ii+1, projection = data_proj)        

            uwnd = ds.u.sel(time = file_date).sel(level = pres_lev)  #zonal winds (m/s)
            uwnd = uwnd.sel(time = uwnd.time.dt.hour.isin(hr))       #zonal winds (m/s)

            vwnd = ds.v.sel(time = file_date).sel(level = pres_lev)  #meridional winds (m/s)
            vwnd = vwnd.sel(time = vwnd.time.dt.hour.isin(hr))       #meridional winds (m/s)
            
            rh = ds.r.sel(time = file_date).sel(level = pres_lev)    #relative humidity (%)
            rh = rh.sel(time = rh.time.dt.hour.isin(hr))             #relative humidity (%)

            ##Smoothing (source: Hannah Zanowski) --> not recommended, see top of document
                ##Metpy smooth_n_point (data to be smoothed, number of points to use in smoothing (5 to 9 are valid), and number of times the smoother is applied)
                    ##see https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.smooth_n_point.html for more info
            #uwnd_smoothed = mpcalc.smooth_n_point(uwnd,9,10)
            #vwnd_smoothed = mpcalc.smooth_n_point(vwnd,9,10)

            if ii == 0:
                #ax.set_title('MIMIC TPW, GPM IMERG, and \n%i hPa Streamlines (%s, %s UTC)' % (pres_lev, file_date, hr2))
                ax.set_title('Low-level (975-925 hPa) Dropsonde SRM Vectors, ERA5 %i hPa RH,\nGPM IMERG, and ERA5 %i hPa Streamlines (%s, %s UTC)' % (pres_lev, pres_lev, file_date, hr2))
            elif ii == 1:
                #ax.set_title('MIMIC TPW, GPM IMERG, and \n%i hPa Streamlines (%s, %s UTC)' % (pres_lev, file_date, hr2))
                ax.set_title('Mid-level (900-700 hPa) Dropsonde SRM Vectors, ERA5 %i hPa RH,\nGPM IMERG, and ERA5 %i hPa Streamlines (%s, %s UTC)' % (pres_lev, pres_lev, file_date, hr2))
            
            ax.set_extent(campaign_extent, ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

            # Add land, coastlines, and borders
            #ax.add_feature(cfeature.LAND, facecolor='0.8')
            ax.coastlines(ls = '-', linewidth = 0.5, color = 'gray')
            
#             #plot MIMIC TPW
#             tpw_levels = np.arange(0, 70.5, 2)
#             pm0 = ax.contourf(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, levels = tpw_levels,
#                               extend = 'max', cmap = cm.jet, transform = data_proj)
# #             pm0 = ax.pcolormesh(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, vmin = 0, vmax = 70,
# #                                 cmap = cm.jet, transform = data_proj, zorder = 0)

            #plot ERA5 RH
            tpw_levels = np.arange(0, 100.1, 2)  #actually RH levels, but keeping the tpw_levels name because we use it elsewhere
            pm0 = ax.contourf(ds.longitude, ds.latitude, rh[0].values, levels = tpw_levels,
                              extend = 'max', cmap = cm.jet, transform = data_proj, alpha = 0.6)
        #             pm0 = ax.pcolormesh(ds.longitude, ds.latitude, rh[0].values, vmin = 0, vmax = 70,
        #                                 cmap = cm.jet, transform = data_proj, zorder = 0)
    
            #plot IMERG Rain Rate
            pm1 = ax.contourf(imerg_lons, imerg_lats, imerg_precip_masked, 
                              levels = np.logspace(np.log10(0.1), np.log10(40), num = len(tpw_levels)), 
                              norm = 'log', extend = 'max', 
                              cmap = cm.jet, transform = data_proj, zorder = 1)
#             pm1 = ax.pcolormesh(imerg_lons, imerg_lats, imerg_precip_masked, 
#                                 norm = mplc.LogNorm(vmin = 0.1, vmax = 40), 
#                                 cmap = cm.jet, transform = data_proj, zorder = 1) 

            #Gridlines
            gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, 
                              linewidth = 0.5, color = 'gray', alpha = 0.5, linestyle = '--', zorder = 2)
            gl.top_labels = False
            gl.right_labels = False
            gl.xlabel_style = {'size':16, 'color':'black'}
            gl.ylabel_style = {'size':16, 'color':'black'}

            #plot ERA5 streamlines
            ax.streamplot(ds.longitude, ds.latitude, uwnd[0].values, vwnd[0].values,
                          color = 'k', linewidth = 0.6, density = 1.0, transform = data_proj, zorder = 3)

            #plot flight track
            ax.plot(flight_lon, flight_lat, color = 'darkmagenta', linewidth = 2.5, zorder = 4)
                
            if ii == 0:    #plot low-level SRM vector
                ax.quiver(drop_data_to_plot[:, 0].astype(float), drop_data_to_plot[:, 1].astype(float), drop_data_to_plot[:, 3].astype(float), drop_data_to_plot[:, 4].astype(float), color = 'k', pivot = 'middle', zorder = 5)
            elif ii == 1:  #plot mid-level SRM vector
                ax.quiver(drop_data_to_plot[:, 0].astype(float), drop_data_to_plot[:, 1].astype(float), drop_data_to_plot[:, 5].astype(float), drop_data_to_plot[:, 6].astype(float), color = 'k', pivot = 'middle', zorder = 5)
            else:
                pass
            
            #plot near-storm dropsonde locations for the given flight 
                #if the dropsonde was deployed within 30 minutes (1-hr total range) of the given hour
                    #NOTE: Dropsondes with no wind data don't have GPS data either (5 of them total)
            for sonde_row in range(drop_data_to_plot.shape[0]):
                if drop_data_to_plot[sonde_row, 0] != None:
                    ax.text(drop_data_to_plot[sonde_row, 0], drop_data_to_plot[sonde_row, 1], drop_data_to_plot[sonde_row, 2], color = 'b', fontsize = 'xx-large', ha = 'center', va = 'top', zorder = 6)
                
            #plotting the colorbars
            #cbar0 = group_fig.colorbar(pm0, ax = ax, orientation = 'vertical', shrink = 0.75, pad = 0.25)
            #cbar0.set_label('TPW [mm]')
            #cbar0.ax.yaxis.set_ticks_position('left')
            #cbar0.ax.yaxis.set_label_position('left')
            
            #this works with GeoAxes (i.e., Cartopy's map projections)
            if ii == len(pressures_to_plot_stream) - 1:
                
#                 #MIMIC TPW colorbar
#                 ticks_tpw = np.arange(0, 70.5, 10, dtype = int)
#                 #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
#                 cax0 = group_fig.add_axes([ax.get_position().x1 + 0.06, ax.get_position().y0, 0.02, 0.755])
#                 cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_tpw)
#                 cbar0.set_label('TPW [mm]')
#                 cbar0.ax.set_yticklabels(list(map(str, list(ticks_tpw))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
#                 cbar0.ax.yaxis.set_ticks_position('left')
#                 cbar0.ax.yaxis.set_label_position('left')

                #ERA5 RH colorbar
                ticks_rh = np.arange(0, 100.5, 10, dtype = int)
                #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
                cax0 = group_fig.add_axes([ax.get_position().x1 + 0.06, ax.get_position().y0, 0.02, 0.755])
                cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_rh)
                cbar0.set_label('Relative Humidity [%]')
                cbar0.ax.set_yticklabels(list(map(str, list(ticks_rh))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
                cbar0.ax.yaxis.set_ticks_position('left')
                cbar0.ax.yaxis.set_label_position('left')
                
                #IMERG colorbar
                ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40], dtype = float)
                cax1 = group_fig.add_axes([ax.get_position().x1 + 0.06, ax.get_position().y0, 0.02, 0.755])
                cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
                cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
                cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
                cbar1.ax.yaxis.set_ticks_position('right')
                cbar1.ax.yaxis.set_label_position('right')
                
        ds_tpw.close()
        ds_imerg.close()                
                
        #plt.tight_layout()
        #plt.subplots_adjust(wspace = 0.1)

        #save the figure
        plot_save_name = f'{file_date}_{hr2}UTC_SRM_streamlines_RH.png'
        plt.savefig(os.path.join('/Users/brodenkirch/Desktop/CPEX-CV_StormRelative_Motion_TIMPS/', plot_save_name), bbox_inches = 'tight')
        #plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
        #plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
        plt.close()
        plt.clf()    #if placing this after plt.close(), may release memory related to the figure (https://stackoverflow.com/questions/741877/how-do-i-tell-matplotlib-that-i-am-done-with-a-plot)

        ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
        ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
        im = Image.open(os.path.join('/Users/brodenkirch/Desktop/CPEX-CV_StormRelative_Motion_TIMPS/', plot_save_name))

        try:
            im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
        except:
            #use this for older version of PIL/Pillow if the above line doesn't work, 
            #though this line will have isolated, extremely minor image effects due to 
            #only using 256 colors instead of the 3-element RGB scale
            im2 = im.convert('P')

        im2.save(os.path.join('/Users/brodenkirch/Desktop/CPEX-CV_StormRelative_Motion_TIMPS/', plot_save_name))
        im.close()
        im2.close()

    ds.close()
    
    print (file_date + ' streamline plots complete!\n')

tend = time.time()

print (f'This script took {np.round((tend - tstart) / 60, 1)} minutes to complete.')


20220926 streamline plots in progress...


/Users/brodenkirch/anaconda3/envs/cpex/lib/python3.9/site-packages/icartt/dataset.py:290: UserWarning: Normal comments: required keyword PLATFORM is missing.
  warnings.warn(
/Users/brodenkirch/anaconda3/envs/cpex/lib/python3.9/site-packages/icartt/dataset.py:692: UserWarning: Number of header lines suggested in line 1 (76) do not match actual header lines read (77)
  warnings.warn(


Skipping 2022-09-26 07:22:22 sonde due to it being "In Precip" (In Precip sondes cannot be inflow sondes)
Skipping 2022-09-26 07:40:43 sonde due to it being "In Precip" (In Precip sondes cannot be inflow sondes)
Skipping 2022-09-26 07:51:33 sonde due to it being "In Precip" (In Precip sondes cannot be inflow sondes)
Skipping 2022-09-26 08:07:04 sonde due to it being "In Precip" (In Precip sondes cannot be inflow sondes)
Skipping 2022-09-26 08:14:02 sonde due to it being "In Precip" (In Precip sondes cannot be inflow sondes)
Skipping 2022-09-26 08:30:06 sonde due to it being "In Precip" (In Precip sondes cannot be inflow sondes)
Skipping 2022-09-26 09:22:10 sonde due to it being "In Precip" (In Precip sondes cannot be inflow sondes)
Skipping 2022-09-26 10:28:13 sonde due to it being "In Precip" (In Precip sondes cannot be inflow sondes)
Skipping 2022-09-26 10:41:56 sonde due to it being "In Precip" (In Precip sondes cannot be inflow sondes)
20220926 streamline plots complete!

This scri

<Figure size 432x288 with 0 Axes>

In [ ]:
# #Calculate the max area (in km^2) of each sampled TIMPS MCS from CPEX-CV

# drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
# df_drop = pd.read_csv(drop_metric_filepath)

# timps_folder = os.path.join(os.getcwd(), 'TIMPS_data')
# timps_areas_dict = {}

# for ii, unique_timps_id in enumerate(df_drop['TIMPS ID'].unique()):
#     timps_filepath = None
#     if pd.isnull(unique_timps_id):
#         continue
#     else:
#         unique_timps_id = str(int(unique_timps_id))
#         for filename in os.listdir(timps_folder):
#             if unique_timps_id in filename:
#                 timps_filepath = os.path.join(timps_folder, filename)
#                 break
    
#     if timps_filepath == None:
#         sys.exit(f'Could not find TIMPS file for TIMP ID {unique_timps_id}')
#     else:
#         timps_ds = xr.open_dataset(timps_filepath)
#         timps_max_area = np.round(timps_ds.area.max().item(), 1)  #max area throughout whole lifetime of the system
#         timps_areas_dict[ii] = [unique_timps_id, timps_max_area]
        
#     timps_ds.close()
        
# timps_areas = pd.DataFrame.from_dict(timps_areas_dict, orient = 'index', columns = ['TIMPS ID', 'Max Area (km$^2$)'])
# timps_areas
